In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
import collections
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
ds = pd.read_csv(r'C:\Users\Sofia\OneDrive - NOVAIMS\Nova IMS\Mestrado\2º semestre\Business Cases\BC2\dataset.csv', sep=";")
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111733 entries, 0 to 111732
Data columns (total 29 columns):
ID                      111733 non-null int64
Nationality             111733 non-null object
Age                     107561 non-null float64
DaysSinceCreation       111733 non-null int64
NameHash                111733 non-null object
DocIDHash               110732 non-null object
AverageLeadTime         111733 non-null int64
LodgingRevenue          111733 non-null float64
OtherRevenue            111733 non-null float64
BookingsCanceled        111733 non-null int64
BookingsNoShowed        111733 non-null int64
BookingsCheckedIn       111733 non-null int64
PersonsNights           111733 non-null int64
RoomNights              111733 non-null int64
DistributionChannel     111733 non-null object
MarketSegment           111733 non-null object
SRHighFloor             111733 non-null int64
SRLowFloor              111733 non-null int64
SRAccessibleRoom        111733 non-null int64
SRMe

In [3]:
# Summary statistics for all variables
summary=ds.describe(exclude=[np.object])   # exclude only objects, that decrease readability and do not bring information
summary=summary.transpose()  # transpose the summary for easier reading
summary.head(len(summary))

,count,mean,std,min,25%,50%,75%,max
ID,111733.0,55867.000000,32254.683151,1.0,27934.0,55867.0,83800.0,111733.00
Age,107561.0,45.639191,17.244952,-10.0,33.0,47.0,58.0,123.00
DaysSinceCreation,111733.0,595.026599,374.657382,36.0,288.0,522.0,889.0,1385.00
AverageLeadTime,111733.0,60.833147,85.115320,-1.0,0.0,21.0,95.0,588.00
LodgingRevenue,111733.0,283.851283,379.131556,0.0,0.0,208.0,393.3,21781.00
OtherRevenue,111733.0,64.682802,123.580715,0.0,0.0,31.0,84.0,8859.25
BookingsCanceled,111733.0,0.002282,0.080631,0.0,0.0,0.0,0.0,15.00
BookingsNoShowed,111733.0,0.000600,0.028217,0.0,0.0,0.0,0.0,3.00
BookingsCheckedIn,111733.0,0.737607,0.730889,0.0,0.0,1.0,1.0,76.00
PersonsNights,111733.0,4.328318,4.630739,0.0,0.0,4.0,6.0,116.00


Questões gerais:
- Faz sentido analisarmos pessoas que não têm qualquer booking feito? Acho que não... têm valor de 0 em quase todas as variáveis.
- Quando é que foram recolhidos os dados? Data
- O que são os GDS Systems? É um distribution channel que junta os voos aos hotéis?
- A informação que temos na altura do booking varia consoante o tipo de channel?

Hash questions:
- É normal algumas pessoas com bookings feitos não terem documento identificação associado? Ser null...
- Pode acontecer que uma empresa faça bookings para os seus colaboradores e o DocIDHash seja igual para todos eles? Há tuplos com o mesmo Doc ID, mas com idades muito diferentes.
- Há um DocIDHash com 3000 observações com idades diferentes e muitos nulls na age, e tem todo o tipo de distribution channel. O que pode querer dizer?
No caso acima não faz sentido fazer merge porque claramente os bookings não dizem respeito todos à mesma pessoa... mas como controlamos isto? -> quando podemos ou não fazer merge.

Variáveis questões:
- O que fazemos com pessoas menores de idade? Que razões existem para eles estarem na nossa base de dados?
- As variáveis binárias são uma média das estadias ou têm a ver com a última estadia? O que fazemos quando fazemos merge de linhas?
- Avg lead time of -1? Means something or is an error? O empregado alterou os dados após a reserva, para corrigir algo?

1as conclusões das estatísticas descritivas:
- Age tem valor mínimo de -10 e máximo de 123 - coherence check
- Avg lead time of -1. Há alguma diferença entre o quartil dos 75% e o valor máximo e a média - possíveis outliers
- Lodging revenue - claros outliers
- Other revenue - claros outliers
- bookings cancelled - check for outliers
- boockings checked in - check for outliers
- persons nights - outliers
- room nights - outliers

Possíveis sugestões de implementação:
- Criar ficha cliente -> para beneficiar os clientes recorrentes.
- Guardar o nome da empresa quando é corporate.

In [4]:
ds[(ds.duplicated(subset=['DocIDHash']))].DocIDHash.value_counts()

0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855    3031
0xA486FBACF4B4E5537B026743E3FDFE571D716839E758236F42950A61FE6B922B      30
0x2B17E9D2CCEF2EA0FE752EE345BEDFB06741FFC8ECECF45D6BBDBAF9A274FF52      23
0x469CF1F9CF8C790FFA5AD3F484F2938CBEFF6435BCFD734F687EC6D1E968F076      14
0x2A14D03A4827C67E0D39408F103DB417AD496DCE6158F8309E6281185C042003      13
                                                                      ... 
0xC2E9F5B7264AF770DB7E78D42C4A566BDAEDA6790C4A2CADD6CDBA30A22B41B1       1
0x50A6908B71943FBF414E6760C7696D61A58CFB29BB55C8392B32CD8E816E2BC6       1
0x6E08496D8485B95A699EDF39A4E5D3D7E6ACA349A23DC2FBB1ACB55DBF8075D3       1
0x6AF1D1958AF5DAEFE05DC55C0224231A5D9DE50F90A8B67B953BCEAA2C0755D0       1
0xCFE5F233866148F95D09F74B1449008412516CAA14380D00D917218B501BE75C       1
Name: DocIDHash, Length: 3085, dtype: int64

In [5]:
a=ds[(ds.duplicated(subset=['DocIDHash']))]

In [6]:
a[(a.DocIDHash=='0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855') & (~a.Age.isnull())]

,ID,Nationality,Age,DaysSinceCreation,NameHash,DocIDHash,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,...,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
744,745,PRT,40.0,1359,0xDE75217D31BF3A07C97BC70DF817FCBA6CC5AA29C153...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,103,388.0,120.0,0,...,0,0,0,0,0,0,0,0,0,0
842,843,PRT,48.0,1355,0x927BA21A63552D1FC65670C559845E2B3C19BCC55B18...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,17,177.0,77.0,0,...,0,0,0,0,0,0,0,0,0,0
907,908,PRT,33.0,1352,0x3CA9DDAB19D8F50A425B14FAB6BA283701AB9B5A651C...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,3,73.0,7.0,0,...,0,0,0,0,0,0,0,0,0,0
1002,1003,PRT,31.0,1348,0x13B95955F49C8E660F3B7639FDAB4BB75A495493A5A2...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,22,172.8,42.0,0,...,0,0,0,0,1,0,0,0,0,0
1003,1004,PRT,30.0,1348,0x155357372FF051877CBE53E7A357935957CB3F22F8D7...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,0,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103345,103346,PRT,23.0,103,0x3B383C03E88F6AFE5508D1E75BBDCB6FE13548B1BDA9...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,1,116.0,16.0,0,...,0,0,0,0,0,1,0,0,0,0
106428,106429,PRT,10.0,78,0x8C93F42F7CE50F07E1B226CDD2111A5413903FA1EC73...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
107858,107859,PRT,6.0,67,0x151FA6F1DCAD61BDCEF01D315AE43293597CE08B0F10...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
108450,108451,PRT,56.0,62,0xD8F7D032DC3955A24CC5394935C8EDC4D06B77812C8A...,0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B...,170,314.0,32.0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
ds[(ds.duplicated(subset=['DocIDHash']))].DocIDHash.value_counts()

0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855    3031
0xA486FBACF4B4E5537B026743E3FDFE571D716839E758236F42950A61FE6B922B      30
0x2B17E9D2CCEF2EA0FE752EE345BEDFB06741FFC8ECECF45D6BBDBAF9A274FF52      23
0x469CF1F9CF8C790FFA5AD3F484F2938CBEFF6435BCFD734F687EC6D1E968F076      14
0x2A14D03A4827C67E0D39408F103DB417AD496DCE6158F8309E6281185C042003      13
                                                                      ... 
0xC2E9F5B7264AF770DB7E78D42C4A566BDAEDA6790C4A2CADD6CDBA30A22B41B1       1
0x50A6908B71943FBF414E6760C7696D61A58CFB29BB55C8392B32CD8E816E2BC6       1
0x6E08496D8485B95A699EDF39A4E5D3D7E6ACA349A23DC2FBB1ACB55DBF8075D3       1
0x6AF1D1958AF5DAEFE05DC55C0224231A5D9DE50F90A8B67B953BCEAA2C0755D0       1
0xCFE5F233866148F95D09F74B1449008412516CAA14380D00D917218B501BE75C       1
Name: DocIDHash, Length: 3085, dtype: int64

In [10]:
ds.DistributionChannel.unique()

array(['Corporate', 'Travel Agent/Operator', 'Direct', 'GDS Systems'],
      dtype=object)

In [12]:
ds[ds.DistributionChannel== 'Corporate'].duplicated(subset=['DocIDHash'])

0         False
374       False
378       False
402       False
405       False
          ...  
111143    False
111225     True
111226     True
111515    False
111601    False
Length: 3135, dtype: bool

In [13]:
ds[(ds.duplicated(subset=['DocIDHash']))&(ds.DistributionChannel== 'Corporate')].DocIDHash.value_counts()

0xE3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855    275
0xA486FBACF4B4E5537B026743E3FDFE571D716839E758236F42950A61FE6B922B     30
0x2B17E9D2CCEF2EA0FE752EE345BEDFB06741FFC8ECECF45D6BBDBAF9A274FF52     22
0x3856085146F7BC27BD07BFC4CA1991ED4E65E179D7BDB7DBBA7E32620809C799     11
0x9220D336F2DDD7B68F5066878889C7637EE28924B249F968F5EC82D895B108A7     11
                                                                     ... 
0xDE497DBCD45593B89D76F9B97CB1B9F3CF9DD677A16998FD35FFB2951A3E8CFD      1
0xD03C2CC7EB9FE7EB4FD0129C5AAAC23DE56B5124E2D937FF79FF4DDB949C521C      1
0x33B0972B52184C63AF39720B9DD68309776CF43398A8ED97657EFEDC095377C3      1
0xD30CB68082161621A24A5D86CADD8A6BD26814C7F65E49913D1B2D8C1D40B687      1
0x1AFF5611ED6521CEA7D3CA7A2A626B4F2C6AC53799534C00B7A2FD480A439E56      1
Name: DocIDHash, Length: 358, dtype: int64

In [14]:
ds['Total_Bookings']=ds.BookingsCanceled + ds.BookingsNoShowed + ds.BookingsCheckedIn

In [20]:
ds['Total_Revenue']=ds.LodgingRevenue + ds.OtherRevenue

In [15]:
ds.Total_Bookings

0         3
1         1
2         0
3         1
4         0
         ..
111728    0
111729    1
111730    0
111731    1
111732    0
Name: Total_Bookings, Length: 111733, dtype: int64

In [24]:
total = ds[(ds.Total_Bookings==0)]
total

,ID,Nationality,Age,DaysSinceCreation,NameHash,DocIDHash,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,...,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom,Total_Bookings,Total_Revenue
2,3,DEU,32.0,1385,0xDA46E62F66936284DF2844EC4FC542D0DAD780C0EE0C...,0x27F5DF762CCDA622C752CCDA45794923BED9F1B66300...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,5,FRA,52.0,1385,0xD2E3D5BFCA141865669F98D64CDA85AD04DEFF47F8A0...,0x42BDEE0E05A9441C94147076EDDCC47E604DA5447DD4...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,7,JPN,50.0,1385,0x94DB830C90A6DA2331968CFC9448AB9A3CE07D7CFEDD...,0x47E5E4B21585F1FD956C768E730604241B380EDFEA68...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,9,FRA,43.0,1385,0x44BB41EF2D87698E75B6FBB77A8815BF48DAA912C140...,0x6C456E45A78A20BC794137AE326A81D587B6528B3944...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0
10,11,IRL,28.0,1385,0x606584F7B9D6A1905BF9B7ED7B7EBE5DF17DF22FECDF...,0xCA46CD508DB3DB2085374141177E07ECEC4B724AE41C...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111723,111724,ITA,56.0,37,0x82277FA4AD074DA9786E64E977C94FEF1C2E47CD6212...,0x5ED6055421E857EA3496D4C75FA2237ACD353DB1B64B...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0
111725,111726,PAN,60.0,37,0xAA3529E305173010BADE927C7015FA091C3883D257E9...,0xDC47997245A7B9ACF9D4974149D2100CEB00981A2F03...,0,0.0,0.0,0,...,0,0,0,1,1,0,0,0,0,0.0
111728,111729,DEU,31.0,36,0x2F59F0E86596B861C3303585FDB7F090B1B2D70442C5...,0x86AC02B9393E7F58354BA4193D792C07DB91D4BE27C7...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0
111730,111731,BRA,37.0,36,0xEF5F6AAA6120F2AC49179E16FD12477C243A8F4FC504...,0x74B9B5A554F5F660371E56D4CE4C1D4C3DC80AE71D68...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0


In [19]:
summary_total=total.describe(exclude=[np.object])   # exclude only objects, that decrease readability and do not bring information
summary_total=summary_total.transpose()  # transpose the summary for easier reading
summary_total.head(len(summary_total))

,count,mean,std,min,25%,50%,75%,max
ID,33197.0,75481.810796,23666.188706,3.0,59462.0,76182.0,94670.0,111733.0
Age,32072.0,39.966139,20.205140,-6.0,24.0,41.0,56.0,123.0
DaysSinceCreation,33197.0,365.488779,247.751981,36.0,176.0,369.0,493.0,1385.0
AverageLeadTime,33197.0,0.013947,2.015055,0.0,0.0,0.0,0.0,349.0
LodgingRevenue,33197.0,0.007510,1.368273,0.0,0.0,0.0,0.0,249.3
OtherRevenue,33197.0,0.000542,0.098792,0.0,0.0,0.0,0.0,18.0
BookingsCanceled,33197.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BookingsNoShowed,33197.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BookingsCheckedIn,33197.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PersonsNights,33197.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [23]:
total = ds[(ds.Total_Bookings==0)&(ds.Total_Revenue==0)]
total

,ID,Nationality,Age,DaysSinceCreation,NameHash,DocIDHash,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,...,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom,Total_Bookings,Total_Revenue
2,3,DEU,32.0,1385,0xDA46E62F66936284DF2844EC4FC542D0DAD780C0EE0C...,0x27F5DF762CCDA622C752CCDA45794923BED9F1B66300...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,5,FRA,52.0,1385,0xD2E3D5BFCA141865669F98D64CDA85AD04DEFF47F8A0...,0x42BDEE0E05A9441C94147076EDDCC47E604DA5447DD4...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,7,JPN,50.0,1385,0x94DB830C90A6DA2331968CFC9448AB9A3CE07D7CFEDD...,0x47E5E4B21585F1FD956C768E730604241B380EDFEA68...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,9,FRA,43.0,1385,0x44BB41EF2D87698E75B6FBB77A8815BF48DAA912C140...,0x6C456E45A78A20BC794137AE326A81D587B6528B3944...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0
10,11,IRL,28.0,1385,0x606584F7B9D6A1905BF9B7ED7B7EBE5DF17DF22FECDF...,0xCA46CD508DB3DB2085374141177E07ECEC4B724AE41C...,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111723,111724,ITA,56.0,37,0x82277FA4AD074DA9786E64E977C94FEF1C2E47CD6212...,0x5ED6055421E857EA3496D4C75FA2237ACD353DB1B64B...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0
111725,111726,PAN,60.0,37,0xAA3529E305173010BADE927C7015FA091C3883D257E9...,0xDC47997245A7B9ACF9D4974149D2100CEB00981A2F03...,0,0.0,0.0,0,...,0,0,0,1,1,0,0,0,0,0.0
111728,111729,DEU,31.0,36,0x2F59F0E86596B861C3303585FDB7F090B1B2D70442C5...,0x86AC02B9393E7F58354BA4193D792C07DB91D4BE27C7...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0
111730,111731,BRA,37.0,36,0xEF5F6AAA6120F2AC49179E16FD12477C243A8F4FC504...,0x74B9B5A554F5F660371E56D4CE4C1D4C3DC80AE71D68...,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,0,0.0


In [22]:
summary_total=total.describe(exclude=[np.object])   # exclude only objects, that decrease readability and do not bring information
summary_total=summary_total.transpose()  # transpose the summary for easier reading
summary_total.head(len(summary_total))

,count,mean,std,min,25%,50%,75%,max
ID,33196.0,75481.958459,23666.529882,3.0,59461.5,76183.0,94670.5,111733.0
Age,32072.0,39.966139,20.205140,-6.0,24.0,41.0,56.0,123.0
DaysSinceCreation,33196.0,365.487438,247.755592,36.0,176.0,369.0,493.0,1385.0
AverageLeadTime,33196.0,0.010513,1.915502,0.0,0.0,0.0,0.0,349.0
LodgingRevenue,33196.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
OtherRevenue,33196.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BookingsCanceled,33196.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BookingsNoShowed,33196.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BookingsCheckedIn,33196.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PersonsNights,33196.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
